In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

In [ ]:
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
mpl.rcParams['text.usetex'] = False
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)
mpl.rcParams['figure.dpi'] = 300

# Decision Trees & Random Forests

[Matthew R. Carbone](https://www.bnl.gov/staff/mcarbone) | _Assistant Computational Scientist, Computational Science Initiative, Brookhaven National Laboratory_

In this tutorial, you will learn the fundamentals of decision trees and random/decision forests, which are ensembles of decision trees. We will not focus on the technical details. Instead, you will 
- Learn the general working principles of a decision tree
- Get a brief introduction on how they are trained (note that the optimal algorithm for training a decision tree is [NP-hard](https://en.wikipedia.org/wiki/NP-hardness))
- See a bit of the math behind how its done (note: not required!)
- See how they can work in a real scientific example

For some other resources, we recommend looking at
- https://developers.google.com/machine-learning/decision-forests/decision-trees (see the entire course, which is excellent)
- https://www.mastersindatascience.org/learning/machine-learning-algorithms/decision-tree/

# Palmer Penguins dataset

To start, we'll be working with the [Palmer Penguins dataset](https://www.kaggle.com/datasets/parulpandey/palmer-archipelago-antarctica-penguin-data)!

Gorman KB, Williams TD, Fraser WR (2014) Ecological Sexual Dimorphism and Environmental Variability within a Community of Antarctic Penguins (Genus Pygoscelis). PLoS ONE 9(3): e90081. doi:10.1371/journal.pone.0090081

Yes, the penguins have their own [PyPI module](https://github.com/mcnakhaee/palmerpenguins)!

In [ ]:
!pip install palmerpenguins

In [ ]:
from palmerpenguins import load_penguins
penguins = load_penguins()

The `penguins` object is just a [Pandas](https://pandas.pydata.org) dataframe. Let's set the objective of our exercise as **predicting the species of penguin** from a subset of the available information. To start, let's simply examine the data and see how many unique species there are.

In [ ]:
penguins.head()

In [ ]:
penguins["species"].unique()

Looks like there's 3! There are also some rows in which we do not have any data. We want to drop those using the `dropna()` method:

In [ ]:
penguins = penguins.dropna()
penguins.head()

Let's plot a few histograms of the data, resolved by the penguin species, to get a feeling of what's going on.

In [ ]:
features = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]
L = len(features)
bins = 15
fig, axs = plt.subplots(1, L, figsize=(2 * L, 1), sharey=True)

for ax, feature in zip(axs, features):

    gentoo_feature = penguins[penguins["species"] == "Gentoo"][feature]
    adelie_feature = penguins[penguins["species"] == "Adelie"][feature]
    chinstrap_feature = penguins[penguins["species"] == "Chinstrap"][feature]

    ax.hist(gentoo_feature, label="Gentoo", bins=bins)
    ax.hist(adelie_feature, alpha=0.5, label="Adelie", bins=bins)
    ax.hist(chinstrap_feature, alpha=0.5, label="Chinstrap", bins=bins)
    ax.set_xlabel(feature)

axs[0].set_ylabel("Counts")
ax.legend(fontsize=6, frameon=False)

plt.show()

Clearly, there are some "decision boundaries" that can differentiate the different species of penguin! In fact, it appears that all four of the features plotted above exhibit some sort of boundary. `bill_length_mm` can differentiate between Adelie and everything else, and the other three can differentiate between Gentoo and everything else.

# Entropy and information

In order to proceed and train a decision tree, it is important to discuss the concept of entropy and information. The information-theoretic [entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) is a measure of the amount of "surprise" in a variable or dataset.

## Intuitive explaination

"Surprise" in this context means how likely it is _you_ are to be surprised by the outcome of a random variable or a "column" in a dataset. Let's consider the following example cases:
- If you have a random variable that can only take on a single value, then the entropy of that random variable is minimized and equal to 0. There is no "surprise" since the outcome is deterministic.
- If you have a random variable with two possible outcomes each equally likely, then the entropy of that random variable is maximized and equal to 1.
- If you have a random variable that can take on many values, each with equal probabilities, then the entropy of that random variable is maximized.

There are, of course, many values of entropy in between. Entropy is given by the equation in the "Mathematical explaination" section.

## Mathematical explaination (optional)

The equation for the information-theoretic entropy is

$$ S(X) = -\sum_{x \in \mathcal{X}} p(x) \log_2 p(x),$$

where $p(x)$ is the probability of class $x$ occuring in a dataset or the probability of observing $x$ in some candidate set $\mathcal{X}.$ Formally, $X$ is a random variable. This is a bit confusing, but just remember that the sum goes over unique possible outcomes. So, in the case of flipping a coin, we can have heads or tails, and thus there are two terms in the sum (which for a fair coin, are equal).

What happens if the probability of observing some $x^\star = 1.$ Necessarily that means all other values for $x$ have $p(x) = 0$ for all $x \neq x^\star.$ This means that the entropy $S(X)=0,$ as there is no "surprise" factor. The only observed quantity is $x^\star$ with probability 1.

Let's consider now the case where $\mathcal{X} = \{x_1, x_2\}$ and $p(x_1)=p(x_2)=0.5.$ This is the case when entropy is maximized. Let's show this quickly using `scipy.minimize` under the constraint that of course all probabilities sum to 1. Note that minimizing $-S(X)$ is equivalent to maximizing $S(X).$ Note as well that this is general: $S(X)$ is maximized when $p(x_i) = p(x_j)$ for all $x_i, x_j \in X.$ We show this as a func exercise below.

In [ ]:
from scipy.optimize import minimize, LinearConstraint

In [ ]:
def entropy(p):
    return -(p * np.log2(p)).sum()

def negative_entropy(p):
    return -entropy(p)

In [ ]:
np.random.seed(123)
set_size = 2
epsilon = 1e-4
bounds = [(epsilon, 1.0-epsilon) for _ in range(set_size)]
coefs = [1.0 for _ in range(set_size)]
x0 = np.random.random(set_size)

In [ ]:
res = minimize(negative_entropy, x0, bounds=bounds, constraints=LinearConstraint(coefs, lb=1.0, ub=1.0))

In [ ]:
res.x

### Mathematical proof of entropy maximization (optional)

[Jensen's Inequality](https://en.wikipedia.org/wiki/Jensen%27s_inequality) states that

$$ f(\mathbb{E}[X]) \leq \mathbb{E}[f(X)].$$

Applying this to the formula for entropy, we have

$$ S(X) = -\sum_{x \in \mathcal{X}} p(x) \log_2 p(x) = \mathbb{E}[-\log_2 p(x)] \leq \log_2 \mathbb{E}[1/p(x)] = \log_2 |\mathcal{X}|.$$

The upper bound is achieved when $p(x)$ satisfies a uniform distribution!

Source: https://en.wikipedia.org/wiki/Entropy_(information_theory)#Further_properties

## Entropy and information of the Penguins!

So how does this idea of entropy apply to the Penguins dataset? Let's take the original database $D$ and partition it by the value of the `bill_depth_mm` column:
- A subset where `bill_depth_mm <= t`, called $D_1$
- A subset where `bill_depth_mm > t`, called $D_2$

The variable $t \in \mathbb{R}$ is in general any real number, but we can constrain it for the sake of argument by the minimum and maximum values of `bill_depth_mm`. We will try this partitioning for many values of `t`. For each subset, we can calculate the [information gain](https://developers.google.com/machine-learning/decision-forests/binary-classification) of the split. This can be thought of loosely as how well the partitioning discriminates between the target values as a function of $t.$ Particularly, the information gain $I$ is given by

$$I(D, D_1, D_2) = S(D) - \frac{|D_1|}{|D|}S(D_1) - \frac{|D_2|}{|D|}S(D_2).$$

Importantly, in this case, the entropy is evaluated on the _target column_. So while the partitioning of $D_i$ is done using `t`, the entropies are evaluated on the _target_.

In [ ]:
def series_entropy(series):
    """Evalutes the entropy of a pd.Series [of categorical data]."""
    
    unique = np.unique(series)
    probs = [(series == uu).mean() for uu in unique]
    return entropy(np.array(probs))

def get_possible_t_values(series):
    """Gets the minimum number possible values for t."""
    
    unique = np.unique(series)
    diff = np.diff(unique)
    return unique[:-1] + diff / 2.0

def information_gain_continuous_column_split(
    df, target_column_name="species", splitting_column_name="bill_depth_mm"
):
    """Computes the information gain for each of the splits given by t."""
    
    # Compute the original entropy S(X)
    series = df[splitting_column_name]
    t_array = get_possible_t_values(series)
    target_column = df[target_column_name]
    original_series_entropy = series_entropy(target_column)
    L = len(df.index)
    
    # Compute the entropy of the new partitions, weighted by their sizes
    ig_values = []
    for t in t_array:
        
        df1 = df[df[splitting_column_name] <= t]
        L1 = len(df1.index)
        df2 = df[df[splitting_column_name] > t]
        L2 = len(df2.index)
        
        d1_entropy = series_entropy(df1[target_column_name])
        d2_entropy = series_entropy(df2[target_column_name])
        
        ig = original_series_entropy - L1 / L * d1_entropy - L2 / L * d2_entropy

        ig_values.append(ig)
    
    return t_array, np.array(ig_values)

def print_ig_info(
    df=penguins,
    splitting_column_name="flipper_length_mm",
    target_column_name="species"
):

    t_array, ig_array = information_gain_continuous_column_split(
        df,
        target_column_name=target_column_name,
        splitting_column_name=splitting_column_name
    )
    argmax = np.argmax(ig_array)
    split_location = t_array[argmax]
    max_ig = ig_array[argmax]
    print(
        f"The splitting location occurs at {splitting_column_name}=={split_location:.02f}, "
        f"with IG=={max_ig:.02f}"
    )

Iterating through the numerical columns `splitting_column_name` can show us where each split is made, and what the information gain is for making the split. Keep this in mind for the next section: it looks like the first split _should_ be for the `flipper_length_mm` column at a value of roughly 206.

In [ ]:
for col in ["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]:
    print_ig_info(splitting_column_name=col)

# Training a single decision tree

The details of training a decision tree efficiently are a bit beyond the scope of this tutorial. However, we can clearly demonstrate the general methodology without loss of value. The general ideas we will present are essentially the same tools used to train decision trees in production-level algorithms. We'll follow along with the tutorial presented [here](https://developers.google.com/machine-learning/decision-forests/practice) (which is also highly recommended!), but we'll use `sklearn` instead of the TensorFlow version.

We first actually want to do some data processing. First, we should make sure our targets are categorical.

In [ ]:
target_column = "species"
classes = np.unique(penguins[target_column]).tolist()
penguins[target_column] = penguins[target_column].map(classes.index)

In [ ]:
print("Unique species are now", np.unique(penguins["species"]))

Next, we need to split our dataset into testing and training.

**WARNING**: in general, you want to do cross validation and hyperparameter tuning (so you would do a testing, _validation_ and training split). See [Jackson Lee's excellent tutorial](https://indico.bnl.gov/event/18154/), which covers this, for an explanation if you haven't already.

For now though, we will not be hyperparameter tuning, just for the sake of time.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
pd_train, pd_test = train_test_split(penguins, test_size=0.1, random_state=123)

For features, we'll select the following subset of the columns (keeping the features numerical instead of categorical for simplicity):
- `bill_length_mm`
- `bill_depth_mm`
- `flipper_length_mm`
- `body_mass_g`

In [ ]:
cols = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]
X_train = pd_train[cols].to_numpy()
X_test = pd_test[cols].to_numpy()
y_train = pd_train[target_column].to_numpy()
y_test = pd_test[target_column].to_numpy()

And now we can initialize, and train, our model.

In [ ]:
classifier = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=123)
classifier.fit(X=X_train, y=y_train)

We can even visualize the tree! More ways to do this [here](https://mljar.com/blog/visualize-decision-tree/). This is a very insightful way to gain some information about how the model is making decisions.

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = plot_tree(
    classifier, 
    feature_names=cols,  
    class_names=target_column,
    filled=True
)

Note where the first split occurs! It might be slightly different since this splitting is evaluated on the training set and not the entire dataset, but it's close to what we found before, confirming our intuition.

This splitting continues _recursively_ until no more splits can be performed.

# Random Forests

A [Random Forest](https://developers.google.com/machine-learning/decision-forests/random-forests) is an ensemble of more than one decision tree (usually some reasonably large number, like 100). Why do we need more than one? What's the point?

Consider the principle of [wisdom of the crowd](https://en.wikipedia.org/wiki/Wisdom_of_the_crowd). A single random person's prediction about just about anything is not robust. However, the aggregate opinion of many independent predictions is much more robust. As long as each individual's estimation is not _terrible_ (or random), it is generally the case that the aggregate opinion is more accurate than any individual's.

Consider this [fun example](https://www.heart.co.uk/showbiz/tv-movies/who-wants-to-be-a-millionaire-axe-ask-the-audience/):

![image](https://imgs.heart.co.uk/images/162692?crop=16_9&width=660&relax=1&signature=bNYg8mltOU1mhoIkDtebHq_wdTs=)

If only a single person were asked this question, your confidence is not going to be nearly as high as if the entire audience (hundreds of people) all agree! (Aside: how do 2% of people think Christmas happens on the last day of the month every year??? Imagine if _that_ was the estimator you queried!)

Random Forests, and other ensemble models, all operate on this principle. The opinion of a diverse set of (mostly) independent estimators is far more reliable than the opinion of any single estimator.

Each estimator in a Random Forest is a decision tree. The way that each tree is trained is very similar to what we've already discussed, though choosing which tree gets which subset of the data, or subset of the features, is an interesting question. See the links above for more details on this!

# Real research example

We now follow along with the manuscript Torrisi _et al_ to demonstrate how a random forest was used in a somewhat recent, highly cited research work. The data we pull below is available [open access](https://data.matr.io/4/).

S. B. Torrisi, M. R. Carbone, B. A. Rohr, J. H. Montoya, Y. Ha, J. Yano, S. K. Suram & L. Hung. [Random forest machine learning models for interpretable X-ray absorption near-edge structure spectrum-property relationships.](https://www.nature.com/articles/s41524-020-00376-6) npj Comput. Mater. 6, 109 (2020).

First, we have to get the data. To do this, we use the `requests` module to directly pull the content of the webpage, and then parse that specific format (which despite the extension is not exactly JSON). It is not important to understand the particulars.

In [ ]:
import json
import requests

In [ ]:
url = "https://s3.amazonaws.com/publications.matr.io/4/deployment/data/files/spectral_data/Ti_XY.json"
r = requests.get(url)
text = r.text.split("\n")
data = [json.loads(xx) for xx in text[:-1]]

Get the inputs and outputs from this list of dictionaries.

In [ ]:
e_grid = data[0]["E"]
spectra = np.array([
    dat["mu"] for dat in data
    if dat["one_hot_coord"] is not None
])
coordinations = np.array([
    dat["coordination"] for dat in data
    if dat["one_hot_coord"] is not None
])

Like the Palmer Penguins dataset, we have reduced our classification task to a 3-class classification problem. In this case, it is the coordination number of an X-ray-absorbing atom! If you don't know what this means, don't worry too much about it. We're simply trying to classify whether or not an X-ray absorption spectrum can be used to predict this number.

In [ ]:
np.unique(coordinations)

Here are what some of the spectra look like. These are our input features, while the classes above are our targets.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2, 1))

for spec in spectra[::50]:
    ax.plot(e_grid, spec, color="black", alpha=0.1)

ax.set_ylabel("$\mu(E)$ / a.u.")
ax.set_yticks([])
ax.set_xlabel("$E$ / e.V.")
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(spectra, coordinations, test_size=0.1)

In [ ]:
rfc = RandomForestClassifier(n_jobs=6, n_estimators=150, random_state=123)
rfc.fit(X_train, y_train)

Note how quickly the random forest trains. Generally speaking, it would be much more expensive in many situations to train e.g. a deep neural network (though in this case there are not many training examples).

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2, 2))
predictions = rfc.predict(X_test)
cm = confusion_matrix(y_test, predictions, labels=rfc.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rfc.classes_)
disp.plot(ax=ax)

## Exercises

Here are some fun topics/questions for you to explore/try to answer. If you have any questions, please feel free to email me!

- Prove that the upper bound for maximizing the entropy is satisfied when $p(x) = 1/n,$ where $n$ is the number of unique possible outcomes of a random variable.
- Investigate [pruning](https://developers.google.com/machine-learning/decision-forests/overfitting-and-pruning), which is a method for reducing overfitting.
- Hard: evaluate the runtime complexity of training a decision tree. Answer [here](https://developers.google.com/machine-learning/decision-forests/binary-classification).